In [74]:
import numpy as np
import pandas as pd

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns

# Загрузка датасета №1 и его предобработка

In [75]:
df = pd.read_excel('datasets/coronavirus_Mar31.xlsx')
df.set_index("EVENT_ID_CNTY", inplace = True)
df['EVENT_DATE'] = pd.to_datetime(df['EVENT_DATE'])
df.head()

,EVENT_DATE,YEAR,TIME_PRECISION,DISORDER_TYPE,EVENT_TYPE,SUB_EVENT_TYPE,ACTOR1,ASSOC_ACTOR_1,INTER1,ACTOR2,...,LOCATION,LATITUDE,LONGITUDE,GEO_PRECISION,SOURCE,SOURCE_SCALE,NOTES,FATALITIES,TAGS,TIMESTAMP
EVENT_ID_CNTY,,,,,,,,,,,,,,,,,,,,,
XKX325,2020-03-13,2020,1,Strategic developments,Strategic developments,Change to group/activity,Government of Kosovo (2020-),NaN,1,NaN,...,Pristina,42.667,21.172,3,Prishtina Insight,National,"On 13 March 2020, the government of Kosovo ena...",0,NaN,1585075531
XKX326,2020-03-19,2020,1,Demonstrations,Protests,Peaceful protest,Protesters (Kosovo),NaN,6,NaN,...,Pristina,42.667,21.172,1,RFE/RL,International,"On 19 March 2020, residents of Pristina, Kosov...",0,crowd size=no report,1585673085
XKX327,2020-03-20,2020,1,Demonstrations,Protests,Peaceful protest,Protesters (Kosovo),NaN,6,NaN,...,Pristina,42.667,21.172,1,Kosovo Online,National,"On 20 March 2020, residents of Pristina, Kosov...",0,crowd size=no report,1585673085
XKX328,2020-03-21,2020,1,Demonstrations,Protests,Peaceful protest,Protesters (Kosovo),NaN,6,NaN,...,Pristina,42.667,21.172,1,Sinjali,National,"On 21 March 2020, citizens took to their balco...",0,crowd size=no report,1585673076
XKX329,2020-03-21,2020,1,Demonstrations,Protests,Peaceful protest,Protesters (Kosovo),NaN,6,NaN,...,Peja,42.659,20.288,1,Sinjali,National,"On 21 March 2020, citizens took to their balco...",0,crowd size=no report,1585673076


Датасет содержит наблюдения о беспорядках, столкновениях и иных конфликтах, непросредственно связанных с эпидемией COVID-19.
## Предобработка 1

In [76]:
df.value_counts(subset = ["DISORDER_TYPE", "EVENT_TYPE", "SUB_EVENT_TYPE"]) #Посмотрим на типы событий

DISORDER_TYPE                       EVENT_TYPE                  SUB_EVENT_TYPE                    
Demonstrations                      Protests                    Peaceful protest                      56930
                                    Riots                       Violent demonstration                  2658
                                    Protests                    Protest with intervention              2046
Political violence                  Riots                       Mob violence                           1529
Strategic developments              Strategic developments      Change to group/activity                882
Political violence                  Violence against civilians  Attack                                  851
Strategic developments              Strategic developments      Other                                   254
                                                                Looting/property destruction            176
                                     

Видим, что подавляющее большинство событий - это мирные протесты. Рассмотрим данные по локациям и действующим лицам.

In [77]:
df[['COUNTRY', 'ADMIN1', 'ADMIN2', 'ADMIN3', 'LOCATION']].nunique()

COUNTRY       220
ADMIN1       2022
ADMIN2       7112
ADMIN3       3669
LOCATION    13999
dtype: int64

In [78]:
df[['ACTOR1', 'ASSOC_ACTOR_1', 'ACTOR2', 'ASSOC_ACTOR_2']].nunique()

ACTOR1            946
ASSOC_ACTOR_1    6639
ACTOR2            611
ASSOC_ACTOR_2    1037
dtype: int64

In [79]:
df[['DISORDER_TYPE', 'EVENT_TYPE', 'SUB_EVENT_TYPE']].nunique()

DISORDER_TYPE      4
EVENT_TYPE         6
SUB_EVENT_TYPE    19
dtype: int64

Становится ясно, что никакого OneHotEncoding по геоданным и акторам не будет, да вообще эти данные кроме визуализаций и присоединения других показателей не очень то нужны.
А вот типы событий имеет смысл OneHot'нуть.

In [80]:
df.value_counts(subset = ["REGION"]) #Посмотрим на распределение наблюдений по регионам

REGION                   
Europe                       25071
North America                10796
South America                 8482
South Asia                    7807
Middle East                   3121
East Asia                     2922
Northern Africa               1574
Southeast Asia                1293
Central America               1062
Caucasus and Central Asia      696
Oceania                        637
Eastern Africa                 624
Caribbean                      584
Southern Africa                440
Western Africa                 413
Middle Africa                  194
dtype: int64

Видим, что больше половины наблюдений приходится на Европу и Северную Америку. Сравним Китай и США

In [81]:
print(df[df["COUNTRY"] == "China"].value_counts(subset = ["COUNTRY"]))
print(df[df["COUNTRY"] == "United States"].value_counts(subset = ["COUNTRY"]))

COUNTRY
China      629
dtype: int64
COUNTRY      
United States    7966
dtype: int64


Есть некоторые подозрения по поводу количества наблюдений в полутрамиллиардном Китае. Посмотрим на источники.

In [82]:
print("КИТАЙ: \n", df[df["COUNTRY"] == "China"].value_counts(subset = ["SOURCE_SCALE", "SOURCE"]), "\n")
print("США: \n", df[df["COUNTRY"] == "United States"].value_counts(subset = ["SOURCE_SCALE", "SOURCE"]))

КИТАЙ: 
 SOURCE_SCALE            SOURCE                          
Subnational             Inmediahk.net                       79
                        HK01                                65
                        Wen Wei Po                          55
International           Radio Free Asia                     55
Other                   CLB (China)                         40
                                                            ..
Other-Subnational       China Aid; Inmediahk.net             1
                        Apple Daily Hong Kong; China Aid     1
Other-New media         Twitter; CLB (China)                 1
National-International  Deutsche Welle; HK01                 1
Other-National          Weiquanwang; RTHK                    1
Length: 134, dtype: int64 

США: 
 SOURCE_SCALE          SOURCE                                                                  
New media             Twitter                                                                     127
Other    

Ожидаемо, данные для США берут из открытых внутренних источников, данные для Китая - из внешних, до которых доходит гораздо меньше событий.

In [83]:
df.value_counts(subset = ['TIMESTAMP'])

TIMESTAMP 
1643149412    361
1677617817    303
1631575313    275
1680633575    263
1642536651    254
             ... 
1618498097      1
1618498094      1
1618498091      1
1618498090      1
1618499525      1
Length: 9665, dtype: int64

Не очень то нужные значения, их даже как идентификатор события не выйдет использовать.

In [84]:
df["TAGS"].value_counts()

crowd size=no report                  36035
crowd size=dozens                      1711
crowd size=hundreds                    1455
crowd size=around 100                  1013
crowd size=around 50                    771
                                      ...  
crowd size=around 2,100                   1
crowd size=around 4,700                   1
crowd size=around 1,412                   1
crowd size=hundreds-more than 1000        1
crowd size=unkown                         1
Name: TAGS, Length: 3128, dtype: int64

Интерпретировать теги очень сложно, поэтому тоже отброшу.

In [85]:
df["ASSOC_ACTOR_1"].value_counts()

Labor Group (Italy)                                                                                      1036
Students (United States)                                                                                  836
Labor Group (Spain)                                                                                       831
Labor Group (United States)                                                                               777
Labor Group (India)                                                                                       776
                                                                                                         ... 
Students (India); AISA: All India Students Association; SFI: Students Federation of India                   1
NSUI: National Students Union of India; JKNC: Jammu and Kashmir National Conference; Students (India)       1
Government of Pakistan (2018-2022); SP: Samajwadi Party                                                     1
Farmers (I

Видим очевидную проблему - к "неорганизованным" объединениям присоединены названия стран, да еще и другие участники. Разделю их, чтобы можно было оценивать событие по участникам.

In [86]:
df['ACTOR1_TYPES'] = df["ASSOC_ACTOR_1"].fillna(df["ACTOR1"])
def process_string(row):
    new_string = row['ACTOR1_TYPES'].replace(' (' + row['COUNTRY'] + ')', '')
    return [new_string]
df['ACTOR1_TYPES'] = df.apply(lambda row: pd.Series(process_string(row)), axis=1)

А теперь сделаем признак "неорганизованность" - если событие не организовано формальным объединением, то признак равен 1, и наоборот.

In [87]:
df_actorcheck = df["ACTOR1_TYPES"].str.split(",", expand = True)
#Ниже список акторов, не являющихся организациями
my_list = ["Protesters", "Labor Group", "Health Workers", "Students", "Rioters", "Teachers", "Farmers", "Women", "Prisoners", "Taxi/Bus Drivers", "Lawyers", "Taxi Drivers", "No Vax", "Muslim Group", "Fishers", "Journalists", "Street Traders", "Orthodox Christian Group", "Protestant Christian Group", "Haredi Jewish Group", "Judges", "Refugees/IDPs", None]
def check_list(row):
    return int(all(elem in my_list for elem in row))

df['UNORGANIZED'] = df_actorcheck.apply(lambda row: check_list(row), axis=1)
df["UNORGANIZED"].value_counts()

1    43124
0    22592
Name: UNORGANIZED, dtype: int64

Примерно две трети событий оказались неорганизованными, это не очень хорошо - у нас дисбаланс классов.

In [88]:
df

,EVENT_DATE,YEAR,TIME_PRECISION,DISORDER_TYPE,EVENT_TYPE,SUB_EVENT_TYPE,ACTOR1,ASSOC_ACTOR_1,INTER1,ACTOR2,...,LONGITUDE,GEO_PRECISION,SOURCE,SOURCE_SCALE,NOTES,FATALITIES,TAGS,TIMESTAMP,ACTOR1_TYPES,UNORGANIZED
EVENT_ID_CNTY,,,,,,,,,,,,,,,,,,,,,
XKX325,2020-03-13,2020,1,Strategic developments,Strategic developments,Change to group/activity,Government of Kosovo (2020-),NaN,1,NaN,...,21.172,3,Prishtina Insight,National,"On 13 March 2020, the government of Kosovo ena...",0,NaN,1585075531,Government of Kosovo (2020-),0
XKX326,2020-03-19,2020,1,Demonstrations,Protests,Peaceful protest,Protesters (Kosovo),NaN,6,NaN,...,21.172,1,RFE/RL,International,"On 19 March 2020, residents of Pristina, Kosov...",0,crowd size=no report,1585673085,Protesters,1
XKX327,2020-03-20,2020,1,Demonstrations,Protests,Peaceful protest,Protesters (Kosovo),NaN,6,NaN,...,21.172,1,Kosovo Online,National,"On 20 March 2020, residents of Pristina, Kosov...",0,crowd size=no report,1585673085,Protesters,1
XKX328,2020-03-21,2020,1,Demonstrations,Protests,Peaceful protest,Protesters (Kosovo),NaN,6,NaN,...,21.172,1,Sinjali,National,"On 21 March 2020, citizens took to their balco...",0,crowd size=no report,1585673076,Protesters,1
XKX329,2020-03-21,2020,1,Demonstrations,Protests,Peaceful protest,Protesters (Kosovo),NaN,6,NaN,...,20.288,1,Sinjali,National,"On 21 March 2020, citizens took to their balco...",0,crowd size=no report,1585673076,Protesters,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZAM1292,2020-04-12,2020,1,Political violence,Violence against civilians,Attack,Police Forces of Zambia (2011-2021),NaN,1,Civilians (Zambia),...,29.497,1,Zambia Watchdog,National,"On 12 April 2020, Zambian police attacked peop...",0,NaN,1631575315,Police Forces of Zambia (2011-2021),0
ZAM1300,2020-07-24,2020,1,Political violence,Violence against civilians,Attack,Police Forces of Zambia (2011-2021),NaN,1,Civilians (Zambia),...,31.325,1,Zambia Watchdog,National,"On 24 July 2020, a man was hit and injured by ...",0,NaN,1631575315,Police Forces of Zambia (2011-2021),0
ZAM1315,2020-09-26,2020,1,Political violence,Violence against civilians,Attack,Police Forces of Zambia (2011-2021),NaN,1,Civilians (Zambia),...,28.283,1,Mwebantu; Daily Nation (Zambia),National,"On 26 September 2020, a teenage boy was shot a...",1,NaN,1631575311,Police Forces of Zambia (2011-2021),0


## Предобработка 2
Теперь подгружаем второй датасет, в нем ежедневная информация о заболеваемости по странам (и куча лишнего мусора).

In [116]:
df_cov = pd.read_csv("datasets/owid-covid-data.csv")
df_cov['date'] = pd.to_datetime(df_cov['date'])
df_cov

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166321,ZWE,Africa,Zimbabwe,2022-03-01,236871.0,491.0,413.000,5395.0,0.0,1.000,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
166322,ZWE,Africa,Zimbabwe,2022-03-02,237503.0,632.0,416.286,5396.0,1.0,1.143,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
166323,ZWE,Africa,Zimbabwe,2022-03-03,237503.0,0.0,362.286,5396.0,0.0,0.857,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
166324,ZWE,Africa,Zimbabwe,2022-03-04,238739.0,1236.0,467.429,5397.0,1.0,0.714,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN


Во - первых, в датасетах наверняка есть несовпадения по странам. Посмотрим на них

In [90]:
unknown = set(df["COUNTRY"].tolist()) - set(df_cov["location"].tolist())
print(unknown)

{'Czech Republic', 'Bailiwick of Guernsey', 'eSwatini', 'Ivory Coast', 'Bailiwick of Jersey', 'North Korea', 'Saint-Barthelemy', 'Northern Mariana Islands', 'American Samoa', 'Guadeloupe', 'French Guiana', 'Puerto Rico', 'Caribbean Netherlands', 'East Timor', 'Saint Helena, Ascension and Tristan da Cunha', 'Saint-Martin', 'Sint Maarten', 'Guam', 'Martinique', 'Reunion', 'Virgin Islands, U.S.', 'Republic of Congo'}


In [91]:
droplist = list(set(df_cov["location"].tolist()) - set(df["COUNTRY"].tolist()))
set(df_cov["location"].tolist()) - set(df["COUNTRY"].tolist())#Это несовпавшие страны из датасета по заболеваемости

{'Africa',
 'Asia',
 'Bonaire Sint Eustatius and Saba',
 'British Virgin Islands',
 'Comoros',
 'Congo',
 "Cote d'Ivoire",
 'Czechia',
 'Eswatini',
 'Europe',
 'European Union',
 'Faeroe Islands',
 'Greenland',
 'Guernsey',
 'High income',
 'Hong Kong',
 'International',
 'Jersey',
 'Kiribati',
 'Low income',
 'Lower middle income',
 'Macao',
 'Marshall Islands',
 'Micronesia (country)',
 'Niue',
 'North America',
 'Northern Cyprus',
 'Oceania',
 'Palau',
 'Pitcairn',
 'Saint Helena',
 'Sao Tome and Principe',
 'Sint Maarten (Dutch part)',
 'South America',
 'Timor',
 'Tokelau',
 'Tuvalu',
 'Upper middle income',
 'Vatican',
 'World'}

Несостыкованных стран много, и наверняка когда мы начнем добавлять дополнительные данные, с ними возникнет такая же проблема, поэтому применим грязный трюк:

In [92]:
df[["COUNTRY","ISO"]][df["COUNTRY"].isin(unknown)].groupby("COUNTRY").count().sort_values(by = "ISO", ascending = False)

,ISO
COUNTRY,
Czech Republic,246
Guadeloupe,241
Martinique,95
Reunion,69
North Korea,55
French Guiana,49
Saint-Martin,29
Ivory Coast,23
Puerto Rico,20


Выкинем все страны кроме первых (по остальным мало наблюдений), причем у нас нет данных по заболеваемости в Guadeloupe, Martinique, Reunion и тем более в North Korea, так что оставляем только Чехию.

In [93]:
df_cov["location"].replace(to_replace = "Czechia", value = "Czech Republic", inplace = True)
df_cov = df_cov.drop(df_cov[df_cov["location"].isin(droplist)].index)
df_cov

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166321,ZWE,Africa,Zimbabwe,2022-03-01,236871.0,491.0,413.000,5395.0,0.0,1.000,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
166322,ZWE,Africa,Zimbabwe,2022-03-02,237503.0,632.0,416.286,5396.0,1.0,1.143,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
166323,ZWE,Africa,Zimbabwe,2022-03-03,237503.0,0.0,362.286,5396.0,0.0,0.857,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
166324,ZWE,Africa,Zimbabwe,2022-03-04,238739.0,1236.0,467.429,5397.0,1.0,0.714,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN


Соединим датасеты

In [94]:
#создание служебных колонок для присоединения
df_cov["LOCDATE"] = df_cov["location"] + df_cov["date"].astype("str")
df["LOCDATE"] = df["COUNTRY"] + df["EVENT_DATE"].astype("str")

In [95]:
df_data = pd.merge(df, df_cov, on = "LOCDATE", how = "left")

Сразу проверим, где не склеилось

In [96]:
df_data[df_data["location"].isna()]

,EVENT_DATE,YEAR,TIME_PRECISION,DISORDER_TYPE,EVENT_TYPE,SUB_EVENT_TYPE,ACTOR1,ASSOC_ACTOR_1,INTER1,ACTOR2,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,2020-03-13,2020,1,Strategic developments,Strategic developments,Change to group/activity,Government of Kosovo (2020-),NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,2022-05-25,2022,1,Demonstrations,Protests,Peaceful protest,Protesters (Kosovo),Health Workers (Kosovo),6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,2022-07-29,2022,1,Demonstrations,Protests,Peaceful protest,Protesters (Kosovo),Health Workers (Kosovo); Labor Group (Kosovo),6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,2022-10-21,2022,1,Demonstrations,Protests,Peaceful protest,Protesters (Kosovo),PSD: Social Democratic Party of Kosovo,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,2022-03-09,2022,2,Demonstrations,Protests,Peaceful protest,Protesters (Afghanistan),Health Workers (Afghanistan); Labor Group (Afg...,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65657,2020-04-08,2020,1,Political violence,Violence against civilians,Attack,Military Forces of Yemen (2016-) Supreme Polit...,NaN,1,Civilians (Ethiopia),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65658,2020-04-08,2020,1,Political violence,Violence against civilians,Attack,Military Forces of Yemen (2016-) Supreme Polit...,NaN,1,Civilians (Ethiopia),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65659,2020-04-08,2020,1,Political violence,Violence against civilians,Attack,Operation Restoring Hope,Military Forces of Saudi Arabia (2015-) Border...,8,Civilians (Ethiopia),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65660,2020-04-09,2020,1,Strategic developments,Strategic developments,Other,Military Forces of Yemen (2016-) Supreme Polit...,NaN,1,Civilians (Yemen),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Увы, с этими наблюдениями придется расстаться - на них просто нет данных по заболеваемости

In [97]:
df_data = df_data.drop(df_data[df_data["location"].isna()].index)
df_data

,EVENT_DATE,YEAR,TIME_PRECISION,DISORDER_TYPE,EVENT_TYPE,SUB_EVENT_TYPE,ACTOR1,ASSOC_ACTOR_1,INTER1,ACTOR2,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
1,2020-03-19,2020,1,Demonstrations,Protests,Peaceful protest,Protesters (Kosovo),NaN,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-20,2020,1,Demonstrations,Protests,Peaceful protest,Protesters (Kosovo),NaN,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-21,2020,1,Demonstrations,Protests,Peaceful protest,Protesters (Kosovo),NaN,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-21,2020,1,Demonstrations,Protests,Peaceful protest,Protesters (Kosovo),NaN,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-03-21,2020,1,Demonstrations,Protests,Peaceful protest,Protesters (Kosovo),NaN,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65711,2020-04-12,2020,1,Political violence,Violence against civilians,Attack,Police Forces of Zambia (2011-2021),NaN,1,Civilians (Zambia),...,3.1,24.7,13.938,2.0,63.89,0.584,NaN,NaN,NaN,NaN
65712,2020-07-24,2020,1,Political violence,Violence against civilians,Attack,Police Forces of Zambia (2011-2021),NaN,1,Civilians (Zambia),...,3.1,24.7,13.938,2.0,63.89,0.584,NaN,NaN,NaN,NaN
65713,2020-09-26,2020,1,Political violence,Violence against civilians,Attack,Police Forces of Zambia (2011-2021),NaN,1,Civilians (Zambia),...,3.1,24.7,13.938,2.0,63.89,0.584,NaN,NaN,NaN,NaN
65714,2020-10-08,2020,2,Political violence,Riots,Mob violence,Rioters (Zambia),NaN,5,Police Forces of Zambia (2011-2021),...,3.1,24.7,13.938,2.0,63.89,0.584,NaN,NaN,NaN,NaN


# Чистка 1
Удалю ненужные столбцы, которые не помогут при анализе. Для начала те, в которых наблюдений много, но они не нужны, а потом те, в которых данных мало.

In [118]:
useless = [
    'YEAR',
    'NOTES', #Текстовое описание новости
    'SOURCE', #Источник
    'TIME_PRECISION', #Точность определения времени события
    'TIMESTAMP', #Как оказалось, это время внесения наблюдения в таблицу
    'GEO_PRECISION', #Точность географической оценки
    'TAGS', #Плохо сделанные теги
    'ACTOR1', 
    'ASSOC_ACTOR_1', #Действующие лица
    'ACTOR2', 
    'ASSOC_ACTOR_2', 
    'CIVILIAN_TARGETING', #Было ли направленное именно на гражданские лица насилие
    'ADMIN1', 
    'ADMIN2',
    'ADMIN3', #Место действия
    'LOCATION',
    'INTERACTION',
    'ACTOR1_TYPES' #Служебная колонка из пункта выше
]

In [99]:
df_data = df_data.drop(useless, axis = 1)

In [100]:
limit = df_data.shape[0] * 0.05 #поставим порог в 5%, пока что
df_data = df_data.drop(df_data.columns[df_data.isna().sum() > limit], axis = 1)

Сохраним все события в табличку для визуализаций

In [101]:
df_data.to_excel("datasets/disorders.xlsx", encoding='utf-8')

Отдельно сохраним именно протесты (пригодится в будущем)

In [102]:
df_protests = df_data[df_data["DISORDER_TYPE"] == "Demonstrations"]
df_protests.to_excel("datasets/protests.xlsx", encoding='utf-8')

In [103]:
df_data

,EVENT_DATE,DISORDER_TYPE,EVENT_TYPE,SUB_EVENT_TYPE,INTER1,INTER2,ISO,REGION,COUNTRY,LATITUDE,...,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,female_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index
1,2020-03-19,Demonstrations,Protests,Peaceful protest,6,0,0,Europe,Kosovo,42.667,...,NaN,NaN,NaN,9795.834,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-20,Demonstrations,Protests,Peaceful protest,6,0,0,Europe,Kosovo,42.667,...,NaN,NaN,NaN,9795.834,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-21,Demonstrations,Protests,Peaceful protest,6,0,0,Europe,Kosovo,42.667,...,NaN,NaN,NaN,9795.834,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-21,Demonstrations,Protests,Peaceful protest,6,0,0,Europe,Kosovo,42.659,...,NaN,NaN,NaN,9795.834,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-03-21,Demonstrations,Protests,Peaceful protest,6,0,0,Europe,Kosovo,42.638,...,NaN,NaN,NaN,9795.834,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65711,2020-04-12,Political violence,Violence against civilians,Attack,1,7,894,Southern Africa,Zambia,-13.700,...,17.7,2.48,1.542,3689.251,234.499,3.94,3.1,2.0,63.89,0.584
65712,2020-07-24,Political violence,Violence against civilians,Attack,1,7,894,Southern Africa,Zambia,-14.243,...,17.7,2.48,1.542,3689.251,234.499,3.94,3.1,2.0,63.89,0.584
65713,2020-09-26,Political violence,Violence against civilians,Attack,1,7,894,Southern Africa,Zambia,-15.417,...,17.7,2.48,1.542,3689.251,234.499,3.94,3.1,2.0,63.89,0.584
65714,2020-10-08,Political violence,Riots,Mob violence,5,1,894,Southern Africa,Zambia,-15.417,...,17.7,2.48,1.542,3689.251,234.499,3.94,3.1,2.0,63.89,0.584


In [104]:
df_data.isna().sum().sort_values(ascending=False)

female_smokers                     2801
new_deaths_smoothed_per_million    2057
new_deaths_smoothed                2057
reproduction_rate                  1208
new_deaths_per_million              960
new_deaths                          960
new_cases_smoothed                  928
new_cases_smoothed_per_million      928
total_deaths                        851
total_deaths_per_million            851
hospital_beds_per_thousand          742
stringency_index                    635
aged_70_older                       244
human_development_index             241
gdp_per_capita                      228
aged_65_older                       212
diabetes_prevalence                 173
cardiovasc_death_rate               137
population_density                  132
median_age                           97
new_cases                            81
new_cases_per_million                81
life_expectancy                      43
total_cases_per_million               2
total_cases                           2


Поскольку делать всё это еще несколько раз ручками мы не хотим, то
## Забацаем модуль

In [128]:
def dataclear(df,
             df_addition,
             useless_features = None,
             na_limit = 5
             ):
    """
    Предобработка ACLED датасета для анализа.

    Args:
        df(pd.DataFrame): Датафрейм с основными данными (ACLED).
        df_addition(pd.DataFrame): Датафрейм с дополнительными данными.
        useless_features(list): Список ненужных колонок.
        na_limit(int): Процент допустимых пропусков в колонках.
        
    Returns:
        pd.DataFrame: Обработанный датафрейм.
    """
    
    df.set_index("EVENT_ID_CNTY", inplace = True)
    df['EVENT_DATE'] = pd.to_datetime(df['EVENT_DATE'])
    df['ACTOR1_TYPES'] = df["ASSOC_ACTOR_1"].fillna(df["ACTOR1"])
    
    
    def process_string(row):
        new_string = row['ACTOR1_TYPES'].replace(' (' + row['COUNTRY'] + ')', '')
        return [new_string]
    df['ACTOR1_TYPES'] = df.apply(lambda row: pd.Series(process_string(row)), axis=1)
    df_actorcheck = df["ACTOR1_TYPES"].str.split(",", expand = True)
    
    my_list = ["Protesters", "Labor Group", "Health Workers", "Students", "Rioters", "Teachers", "Farmers", "Women", "Prisoners", "Taxi/Bus Drivers", "Lawyers", "Taxi Drivers", "No Vax", "Muslim Group", "Fishers", "Journalists", "Street Traders", "Orthodox Christian Group", "Protestant Christian Group", "Haredi Jewish Group", "Judges", "Refugees/IDPs", None]
    def check_list(row):
        return int(all(elem in my_list for elem in row))
    df['UNORGANIZED'] = df_actorcheck.apply(lambda row: check_list(row), axis=1)
    
    #работа со вторым датасетом
    df_addition['date'] = pd.to_datetime(df_addition['date'])
    df_addition["location"].replace(to_replace = "Czechia", value = "Czech Republic", inplace = True) #оправданный костыль
    
    #стыковка двух датасетов по общему столбцу
    df_addition["LOCDATE"] = df_addition["location"] + df_addition["date"].astype("str")
    df["LOCDATE"] = df["COUNTRY"] + df["EVENT_DATE"].astype("str")
    df_data = pd.merge(df, df_cov, on = "LOCDATE", how = "left")
    
    #очистка ненужных столбцов
    df_data = df_data.drop([
    'YEAR',
    'NOTES', #Текстовое описание новости
    'SOURCE', #Источник
    'TIME_PRECISION', #Точность определения времени события
    'TIMESTAMP', #Как оказалось, это время внесения наблюдения в таблицу
    'GEO_PRECISION', #Точность географической оценки
    'TAGS', #Плохо сделанные теги
    'ACTOR1', 
    'ASSOC_ACTOR_1', #Действующие лица
    'ACTOR2', 
    'ASSOC_ACTOR_2', 
    'CIVILIAN_TARGETING', #Было ли направленное именно на гражданские лица насилие
    'ADMIN1', 
    'ADMIN2',
    'ADMIN3', #Место действия
    'LOCATION',
    'INTERACTION',
    'ACTOR1_TYPES' #Служебная колонка
    ], axis = 1)
    if useless_features != None:
        df_data = df_data.drop(useless_features, axis = 1)
    df_data = df_data.drop(df_data[df_data["location"].isna()].index)
    df_data = df_data.drop(df_data.columns[df_data.isna().sum() > (na_limit * 0.01 * df_data.shape[0])], axis = 1)
    
    
    return(df_data)

Итак, на вход поступает два датафрейма и (опционально) список ненужных колонок и допустимый процент пропусков для колонки.Проверим, работает ли

In [129]:
df = pd.read_excel('datasets/coronavirus_Mar31.xlsx')
df_cov = pd.read_csv("datasets/owid-covid-data.csv")
dataclear(df, df_cov)

,EVENT_DATE,DISORDER_TYPE,EVENT_TYPE,SUB_EVENT_TYPE,INTER1,INTER2,ISO,REGION,COUNTRY,LATITUDE,...,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,female_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index
1,2020-03-19,Demonstrations,Protests,Peaceful protest,6,0,0,Europe,Kosovo,42.667,...,NaN,NaN,NaN,9795.834,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-20,Demonstrations,Protests,Peaceful protest,6,0,0,Europe,Kosovo,42.667,...,NaN,NaN,NaN,9795.834,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-21,Demonstrations,Protests,Peaceful protest,6,0,0,Europe,Kosovo,42.667,...,NaN,NaN,NaN,9795.834,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-21,Demonstrations,Protests,Peaceful protest,6,0,0,Europe,Kosovo,42.659,...,NaN,NaN,NaN,9795.834,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-03-21,Demonstrations,Protests,Peaceful protest,6,0,0,Europe,Kosovo,42.638,...,NaN,NaN,NaN,9795.834,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65711,2020-04-12,Political violence,Violence against civilians,Attack,1,7,894,Southern Africa,Zambia,-13.700,...,17.7,2.48,1.542,3689.251,234.499,3.94,3.1,2.0,63.89,0.584
65712,2020-07-24,Political violence,Violence against civilians,Attack,1,7,894,Southern Africa,Zambia,-14.243,...,17.7,2.48,1.542,3689.251,234.499,3.94,3.1,2.0,63.89,0.584
65713,2020-09-26,Political violence,Violence against civilians,Attack,1,7,894,Southern Africa,Zambia,-15.417,...,17.7,2.48,1.542,3689.251,234.499,3.94,3.1,2.0,63.89,0.584
65714,2020-10-08,Political violence,Riots,Mob violence,5,1,894,Southern Africa,Zambia,-15.417,...,17.7,2.48,1.542,3689.251,234.499,3.94,3.1,2.0,63.89,0.584


Всё работает, результат идентичен ручной обработке, можно переходить к следующему этапу. Он в тетрадке 1.5 - Road to Visualisation.